In [1]:
import asyncio
import logging
import sys
import os

from astropy.coordinates import Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

#import mt_utils

In [2]:
print(os.environ["OSPL_URI"])

file:///opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/config/ospl-shmem-debug.xml


In [3]:
#os.environ["OSPL_URI"]="file:///home/bstalder/ts_ddsconfig/config/ospl-sp.xml"
os.environ["LSST_DDS_RESPONSIVENESS_TIMEOUT"]="15s"

In [4]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [5]:
domain = salobj.Domain()

await asyncio.sleep(10)

#mtcs = MTCS(domain)
comcam = ComCam(domain)
#mtptg = salobj.Remote(domain, "MTPtg")
#newmtmount = salobj.Remote(domain, "NewMTMount")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task) #, mtptg.start_task, newmtmount.start_task)

cccamera: Adding all resources.


Disabling authlist-based command authorization


ccheaderservice: Adding all resources.


Disabling authlist-based command authorization


ccarchiver: Adding all resources.


...

Read 1 history items for RemoteEvent(CCCamera, 0, softwareVersions)


Read 45 history items for RemoteEvent(CCCamera, 0, startIntegration)


Read 45 history items for RemoteEvent(CCCamera, 0, startReadout)


Read 4 history items for RemoteEvent(CCCamera, 0, summaryState)


[[None, None, None]]

In [6]:
await comcam.enable(settings={"cccamera": "Normal",
                              "ccarchiver": "normal",
                              "ccheaderservice": None})

Enabling all components


Gathering settings.


Received settings from users.: {'cccamera': 'Normal', 'ccarchiver': 'normal', 'ccheaderservice': None}


Settings versions: {'cccamera': 'Normal', 'ccarchiver': 'normal', 'ccheaderservice': None}


[cccamera]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]


[ccheaderservice]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]


[ccarchiver]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]


All components in <State.ENABLED: 2>.


In [7]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'
#group_id = "CALSET_20200914_1449"
print(group_id)

CALSET_20220120_1607


In [8]:
#get available filters
await comcam.get_available_filters()

['i_06', 'r_03', 'g_07']

In [9]:
#get current filter
await comcam.get_current_filter()

'r_03'

In [17]:
# set filter manually
#old way
#comcam.rem.cccamera.cmd_setFilter.set(name="i_06")
#await comcam.rem.cccamera.cmd_setFilter.start(timeout=60)
#new way
await comcam.setup_instrument(filter="i_06")

AckTimeoutError: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=1403708384, ack=<SalRetCode.CMD_INPROGRESS: 301>, error=241, result='Ack : OK')

In [25]:
#for testing L3 condensation
await comcam.take_flats(exptime=10.0, nflats=1, group_id=group_id, note="test")#, filter="g_07")

imagetype: FLAT, skip TCS synchronization.


FLAT 0001 - 0001


array([2022011300014])

In [10]:
await comcam.take_bias(nbias=10, group_id=group_id, note="test")

imagetype: BIAS, skip TCS synchronization.


BIAS 0001 - 0010


BIAS 0002 - 0010


BIAS 0003 - 0010


BIAS 0004 - 0010


...

BIAS 0007 - 0010


BIAS 0008 - 0010


BIAS 0009 - 0010


BIAS 0010 - 0010


array([2022012000001, 2022012000002, 2022012000003, 2022012000004,
       2022012000005, 2022012000006, 2022012000007, 2022012000008,
       2022012000009, 2022012000010])

In [21]:
await comcam.take_darks(exptime=100.0, ndarks=10, group_id=group_id, note="standardCals")

imagetype: DARK, skip TCS synchronization.
DARK 0001 - 0010
DARK 0002 - 0010
logMessage DDS read queue is filling: 14 of 100 elements
DARK 0003 - 0010
DARK 0004 - 0010
DARK 0005 - 0010
DARK 0006 - 0010
DARK 0007 - 0010
DARK 0008 - 0010
DARK 0009 - 0010
DARK 0010 - 0010


array([2021091600057, 2021091600058, 2021091600059, 2021091600060,
       2021091600061, 2021091600062, 2021091600063, 2021091600064,
       2021091600065, 2021091600066])

In [20]:
await comcam.take_flats(exptime=15.0, nflats=10, group_id=group_id, note="standardCals")

imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0010
FLAT 0002 - 0010
FLAT 0003 - 0010
FLAT 0004 - 0010
FLAT 0005 - 0010
FLAT 0006 - 0010
logMessage DDS read queue is filling: 10 of 100 elements
FLAT 0007 - 0010
FLAT 0008 - 0010
FLAT 0009 - 0010
FLAT 0010 - 0010


array([2021091600047, 2021091600048, 2021091600049, 2021091600050,
       2021091600051, 2021091600052, 2021091600053, 2021091600054,
       2021091600055, 2021091600056])

In [ ]:
# FOR PTC run
await comcam.take_bias(nbias=5, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=0.5, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=1.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=1.5, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=2.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=3.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=4.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=5.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=6.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=7.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=8.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=9.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=10.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=15.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=20.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=25.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=30.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=40.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=50.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=60.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_bias(nbias=5, group_id=group_id, note="PTC")

In [ ]:
# For gain stability run
await comcam.take_bias(nbias=5, group_id=group_id, note="gainStability")
for i in range(0,2000):
    await comcam.take_flats(exptime=21.0, nflats=2000, group_id=group_id, note="gainStability")
    await asyncio.sleep(10)
    
await comcam.take_bias(nbias=5, group_id=group_id, note="gainStability")

In [7]:
await comcam.standby()

[cccamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]


[ccheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]


[ccarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]


All components in <State.STANDBY: 5>.


In [27]:
await domain.close()